## 더미데이터 + 테스트

#### 세팅값(거의 default값)

In [24]:
# file: test_gpt_graph_summary.py
# -*- coding: utf-8 -*-

import os
import json
from statistics import mean, pstdev
from typing import List, Dict, Any, Optional
import numpy as np

# -----------------------------
# OpenAI (optional)
# -----------------------------
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
try:
    from openai import OpenAI
    _OPENAI_AVAILABLE = True
except Exception:
    OpenAI = None
    _OPENAI_AVAILABLE = False

# -----------------------------
# 분석 함수
# -----------------------------
def _linear_fit(values: List[float]) -> (float, float):
    x = np.arange(len(values), dtype=float)
    y = np.array(values, dtype=float)
    coef = float(np.polyfit(x, y, 1)[0])
    yhat = np.poly1d(np.polyfit(x, y, 1))(x)
    ss_res = float(np.sum((y - yhat) ** 2))
    ss_tot = float(np.sum((y - np.mean(y)) ** 2))
    r2 = 0.0 if ss_tot == 0 else 1.0 - (ss_res / ss_tot)
    return coef, r2

def _volatility(values: List[float]) -> float:
    if len(values) < 2:
        return 0.0
    mu = mean(values)
    if mu == 0:
        return 0.0
    sd = pstdev(values)
    return float(sd / mu)

def _zscore_anomalies(labels: List[str], values: List[float], thresh: float = 2.0) -> List[Dict[str, Any]]:
    if len(values) < 2:
        return []
    mu = mean(values)
    sd = pstdev(values)
    if sd == 0:
        return []
    out = []
    for lab, val in zip(labels, values):
        z = (val - mu) / sd
        if abs(z) >= thresh:
            out.append({"label": lab, "value": float(val), "zscore": float(z), "note": "피크" if z > 0 else "저점"})
    return out

def _trend_label_and_score(slope: float, r2: float, vol: float) -> (str, float):
    if abs(slope) < 1e-8:
        base = "flat"
        score = max(0.0, 1 - vol)
    else:
        base = "up" if slope > 0 else "down"
        score = min(1.0, max(0.0, (abs(slope) / (abs(slope) + 1)) * 0.6 + r2 * 0.4))
    if vol >= 0.15 and base == "flat":
        base = "volatile"
        score *= 0.5
    return base, float(round(score, 3))

def analyze(labels: List[str], values: List[float]) -> Dict[str, Any]:
    slope, r2 = _linear_fit(values)
    vol = _volatility(values)
    anoms = _zscore_anomalies(labels, values, 2.0)
    tlabel, tscore = _trend_label_and_score(slope, r2, vol)
    return {
        "slope": float(slope),
        "r2": float(round(r2, 4)),
        "volatility": float(round(vol, 4)),
        "trend": tlabel,
        "trend_score": float(round(tscore, 3)),
        "anomalies": anoms,
    }

# -----------------------------
# GPT 요약
# -----------------------------
def gpt_summary(metric_name: str, labels: List[str], values: List[float],
                company: str, unit: str, freq: str, ana: Dict[str, Any]) -> Optional[str]:
    # ✅ GPT 키 없으면 건너뜀
    if not OPENAI_API_KEY or not _OPENAI_AVAILABLE:
        return None

    client = OpenAI(api_key=OPENAI_API_KEY)

    # ✅ 프롬프트에 넣을 추가 지표들 계산(최근 변화, 피크/저점)
    last_label = labels[-1]
    last_val = float(values[-1])
    prev_val = float(values[-2]) if len(values) >= 2 else last_val
    delta_abs = last_val - prev_val
    delta_pct = 0.0 if prev_val == 0 else (delta_abs / prev_val) * 100.0

    peak_idx = int(np.argmax(values))
    trough_idx = int(np.argmin(values))
    peak_label, peak_val = labels[peak_idx], float(values[peak_idx])
    trough_label, trough_val = labels[trough_idx], float(values[trough_idx])

    # ✅ 프롬프트 본문 (분석치 + 컨텍스트 반영)
    prompt = f"""다음 시계열 그래프를 한국어로 중학생도 알아볼 수 있을 만큼 쉽고 간단하게 요약해줘.

[컨텍스트]
- 회사: {company or "해당 기업"}
- 지표: {metric_name}
- 단위: {unit or ""}
- 빈도: {freq or ""}

[데이터]
- 라벨: {labels}
- 값: {[float(v) for v in values]}

[사전 분석치]
- 선형 기울기: {ana["slope"]:.4f}
- 결정계수 R^2: {ana["r2"]:.2f}
- 변동성: {ana["volatility"]:.2f}
- 이상치 개수: {len(ana["anomalies"])}

[포인트]
- 최근값: {last_label} = {last_val}
- 직전 대비 변화: {delta_abs:+.2f} ({delta_pct:+.2f}%)
- 최고치: {peak_label} = {peak_val}
- 최저치: {trough_label} = {trough_val}

[지시]
1) 전반적인 흐름과 최근 변화 중심으로 **2~3문장**으로 써라.
2) 숫자 비교는 1~2개만 핵심만 써라(예: "최근 분기 -3%").
3) 과대해석 금지. 불확실하면 '가능성/추정'으로 표현.
4) 이모지·과장 표현·불필요한 기호 금지.
5) 중학생 수준의 쉬운 말로, 이 흐름과 변화가 무엇을 의미하는지 간단한 해석을 덧붙일 것.
"""

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "너는 차트 아래 들어갈 쉽고 객관적인 한국어 요약을 작성한다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=320,
    )
    return (resp.choices[0].message.content or "").strip()


#### 변경값(현재 더미데이터 넣은 상태)

In [25]:
# -----------------------------
# 테스트 실행
# -----------------------------
if __name__ == "__main__":
    # 더미 데이터
    labels = ["2024-Q1", "2024-Q2", "2024-Q3", "2024-Q4", "2025-Q1"] # ★★★labels 값 변경!!!
    values = [120, 135, 142, 160, 155] # ★★★ values 값 변경!!!

    ana = analyze(labels, values)

    # ✅ 실제 값으로 전달(중괄호 플레이스홀더 제거)
    company = "삼성전자"
    unit = "억 원"
    freq = "quarterly"

    summary_text = gpt_summary("매출액", labels, values, company, unit, freq, ana)
    if not summary_text:
        dir_kor = {"up": "상승", "down": "하락", "flat": "횡보", "volatile": "변동성 확대"}[ana["trend"]]
        summary_text = f"전반적으로 {dir_kor} 흐름이 관찰됩니다(R²={ana['r2']:.2f})."
        if ana["anomalies"]:
            a = ana["anomalies"][-1]
            summary_text += f" {a['label']} 시점에 이상치(값 {a['value']})가 있습니다."

    print("\n📊 분석 결과")
    print(json.dumps(ana, ensure_ascii=False, indent=2))
    print("\n📝 요약 결과")
    print(summary_text)


📊 분석 결과
{
  "slope": 9.49999999999999,
  "r2": 0.8803,
  "volatility": 0.1006,
  "trend": "up",
  "trend_score": 0.895,
  "anomalies": []
}

📝 요약 결과
삼성전자의 매출액은 2024년 1분기 120억 원에서 시작해 2024년 4분기에는 160억 원으로 증가했습니다. 하지만 2025년 1분기에는 155억 원으로 줄어들어 최근 분기에서 -3%의 감소가 있었습니다. 이 변화는 매출이 전반적으로 상승하다가 최근에 약간 줄어든 것을 보여주며, 앞으로의 매출 흐름에 대한 주의가 필요할 수 있습니다.
